In [1]:
import json
import new_env

/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("dataset.json") as fp:
    dataset = json.load(fp)


env = new_env.HolEnv("T")
paper_goals = []
for goal in dataset:
    try:
        p_goal = env.get_polish(goal)
        paper_goals.append((p_goal[0]["polished"]['goal'], goal))
    except:
        print (goal)

Importing theories...
Loading modules...
Configuration done.
DATATYPE ((sum :(α -> α + β) -> (β -> α + β) -> γ) (INL :α -> α + β) (INR :β -> α + β))
∀(A :α -> bool) (B :β -> bool) (C :α -> bool) (D :β -> bool). A × B ∩ (C × D) = A ∩ C × (B ∩ D)
DATATYPE ((list :α list -> (α -> α list -> α list) -> bool) ([] :α list) (CONS :α -> α list -> α list))


In [5]:
polished = [i for i in map(lambda x: x[0], paper_goals)]

In [174]:
#[i for i in map(lambda x: (x.value, x._type, x.arity), x)]

[]

In [15]:
class AST:
    def __init__(self, node, children=[]):
        self.node = node
        self.children = children
    def _print(self, depth=1):
        print (depth * "--- " + self.node.value)
        if len(self.children) > 0:
            for child in self.children:
                child._print(depth+1)
        
class Token:
    def __init__(self, value, _type, arity=None):
        self.value = value
        self._type = _type
        self.arity = arity
    
#assume ast has been passed with ast.node as function 
def func_to_ast(ast, tokens, arity):
    if len(tokens) == 0:
        return ast

    node = tokens[0]
    tokens.pop(0)

    new_node = AST(node, children = [])
    
    if node._type == "variable":

        ast.children.append(new_node)

    elif node._type == "func":

        new_ast = func_to_ast(new_node, tokens, node.arity)
        ast.children.append(new_ast)
      
        
    if arity == 1:
        return ast
    else:
        return func_to_ast(ast, tokens, arity-1)
    
def tokens_to_ast(tokens):
    ast = AST(tokens[0], children=[])
    tokens.pop(0)
    return func_to_ast(ast, tokens, ast.node.arity)


In [55]:
def polished_to_tokens_2(goal):
    polished_goal = goal.split(" ")
    tokens = []
    
    while len(polished_goal) > 0:
        if polished_goal[0] == '@':
            polished_goal.pop(0)
            arity = 1
            
            while polished_goal[0] == '@':
                arity += 1
                polished_goal.pop(0)
                
            func = polished_goal[0]
            polished_goal.pop(0)
            
            #quantifiers should be considered with arity 2 for constructing the tree (for now?)?
            if func == 'C$bool$' and (polished_goal[0] == '!' or polished_goal[0] == '?'):
                    arity = 2
                    #as there's 2 space separated characters for quantifiers
                    func = func + ":" + polished_goal[0] + polished_goal[1]
                    polished_goal = polished_goal[2:]
                    
            else:
                if func[0] == 'C':
                    #should only be one string after the library 
                    func = func + ":" + polished_goal[0]
                    polished_goal.pop(0)
                #otherwise variable func, and nothing following it
                
            tokens.append(Token(func, "func", arity))
        
        #variable or constant case
        else:
            var = polished_goal[0]
            polished_goal.pop(0)
            if var[0] == "C":
                #need to append this and the next as constants are space separated
                var = var + polished_goal[0]
                polished_goal.pop(0)
                
            tokens.append(Token("".join(var), "variable"))
            
    return tokens
            

In [58]:
for i in range(10):
    print (paper_goals[i])

    x = polished_to_tokens_2(polished[i])
    test = tokens_to_ast(x)
    

    test._print()
#     print (paper_goals[i][0])
#     print ([i for i in map(lambda y: y.value, x)])

('@ C$bool$ ! | Vs1 @ C$bool$ ! | Vs2 @ @ C$min$ = @ @ C$pred_set$ PSUBSET Vs1 Vs2 @ @ C$bool$ /\\ @ @ C$pred_set$ SUBSET Vs1 Vs2 @ C$bool$ ~ @ @ C$pred_set$ SUBSET Vs2 Vs1', '∀(s1 :α -> bool) (s2 :α -> bool). s1 ⊂ s2 ⇔ s1 ⊆ s2 ∧ ¬(s2 ⊆ s1)')
--- C$bool$:!|
--- --- Vs1
--- --- C$bool$:!|
--- --- --- Vs2
--- --- --- C$min$:=
--- --- --- --- C$pred_set$:PSUBSET
--- --- --- --- --- Vs1
--- --- --- --- --- Vs2
--- --- --- --- C$bool$:/\
--- --- --- --- --- C$pred_set$:SUBSET
--- --- --- --- --- --- Vs1
--- --- --- --- --- --- Vs2
--- --- --- --- --- C$bool$:~
--- --- --- --- --- --- C$pred_set$:SUBSET
--- --- --- --- --- --- --- Vs2
--- --- --- --- --- --- --- Vs1
('@ C$bool$ ! | Vp @ @ C$min$ = @ C$bool$ ? | Vn @ Vp Vn @ @ C$bool$ /\\ @ Vp @ C$while$ LEAST Vp @ C$bool$ ! | Vn @ @ C$min$ ==> @ @ C$prim_rec$ < Vn @ C$while$ LEAST Vp @ C$bool$ ~ @ Vp Vn', '∀(p :num -> bool). (∃(n :num). p n) ⇔ p ($LEAST p) ∧ ∀(n :num). n < $LEAST p ⇒ ¬p n')
--- C$bool$:!|
--- --- Vp
--- --- C$min$:=
--- --- 

In [33]:
# x = Token("x", "func", 2)
# plus = Token("+", "func", 2)
# neg = Token("-", "func", 1)
# two = Token("2", "variable")
# five = Token("5", "variable")
# seven = Token("7", "variable")
# f = Token("f", "func", 3)
# tokens = [x, neg, two, plus, five, f, two, five, seven]

# ast_ = AST(x)

# tokens.pop(0)

# new_ast = func_to_ast(ast_, tokens, 2)

In [173]:
# def polished_to_tokens(goal):
#     polished_goal = goal.replace(" ", "")
#     tokens = []
#     polished_goal = list(polished_goal)
    
#     while len(polished_goal) > 0:
#         if polished_goal[0] == '@':
#             polished_goal.pop(0)
#             arity = 1
#             while polished_goal[0] == '@':
#                 arity += 1
#                 polished_goal.pop(0)
#             func = []
#             #if variable straight after @, indicates variable function
#             if polished_goal[0] == "V":
#                 polished_goal.pop(0)
            
#             while polished_goal[0] != 'V' and polished_goal[0] != '@':
#                 func.append(polished_goal[0])
#                 polished_goal.pop(0)
#             func = "".join(func)
#             #quantifiers should be considered with arity 2 for constructing the tree (for now?)?
#             if func == 'C$bool$!|' or func == 'C$bool$?|':
#                 arity = 2
#             tokens.append(Token(func, "func", arity))
        
#         elif polished_goal[0] == "V":
#             var = []
#             polished_goal.pop(0)
#             while polished_goal[0] != 'V' and polished_goal[0] != '@':
#                 var.append(polished_goal[0])
#                 polished_goal.pop(0)
#                 if len(polished_goal) == 0:
#                     break
#             tokens.append(Token("".join(var), "variable"))
            
            
#         else:
#             print ("confused")
            
#     return tokens
            
            
        